In [ ]:
! pip install pyannote.audio
! pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 44.4 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.0 MB/s eta 0:00:00


In [ ]:
# !huggingface-cli login

In [ ]:
from pyannote.audio import Pipeline
from transformers import pipeline
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
import librosa
import torch
import os
from indicnlp.normalize.indic_normalize import DevanagariNormalizer
# 1. Configure Environment
from google.colab import userdata
hf_token= userdata.get('HF_TOKEN')  # Your HF token

# 2. Initialize Speaker Diarization Pipeline (Search Result 1,3)

diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=True
).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))


# 3. Perform Speaker Diarization (Search Result 7)
audio_file = "/content/videoplayback.wav"
try:
    diarization = diarization_pipeline(audio_file)
except FileNotFoundError:
    raise ValueError(f"Audio file not found: {audio_file}")
except Exception as e:
    raise RuntimeError(f"Diarization failed: {str(e)}")

# 4. Initialize Hindi ASR Components (Search Result 4,9)
# Whisper Model
whisper_asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    device=0 if torch.cuda.is_available() else -1
)
# Text Normalizer
indic_factory = IndicNormalizerFactory()
normalizer = DevanagariNormalizer(
    lang='hi',
    remove_nuktas=False,
    nasals_mode='do_nothing',
    do_normalize_chandras=False,
    do_normalize_vowel_ending=False
)

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
transcript = []
for segment, _, speaker in diarization.itertracks(yield_label=True):
    audio, sr = librosa.load(
        "/content/videoplayback.wav",
        sr=16000,
        offset=segment.start,
        duration=segment.duration,
        mono=True
    )

    try:
        result = whisper_asr(
            audio,
            return_timestamps=True  # Pipeline-specific param
        )
        transcript.append({
            "speaker": speaker,
            "start": float(segment.start),
            "end": float(segment.end),
            "text": result["text"]
        })
    except Exception as e:
        print(f"Failed segment {segment.start}-{segment.end}: {str(e)}")

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Whisper did not predict an ending timestamp, which can happen if audio is cu

In [ ]:
transcript

[{'speaker': 'SPEAKER_03',
  'start': 0.03096875,
  'end': 0.63846875,
  'text': ' qualquer'},
 {'speaker': 'SPEAKER_03',
  'start': 1.1953437500000001,
  'end': 5.464718750000001,
  'text': ' समाच के हित्में हम और एक बहुत अच्छो काम कर वे आये हैं का तमाई अनुमती है?'},
 {'speaker': 'SPEAKER_04',
  'start': 5.920343750000001,
  'end': 8.384093750000002,
  'text': ' हाँ हाँ समाच के हितमें जू काम होए वो करो'},
 {'speaker': 'SPEAKER_04',
  'start': 16.04534375,
  'end': 18.559718750000002,
  'text': ' पानी से मिझ्जोले, जेबार मनो'},
 {'speaker': 'SPEAKER_04',
  'start': 19.35284375,
  'end': 20.21346875,
  'text': '看了一天'},
 {'speaker': 'SPEAKER_04',
  'start': 22.89659375,
  'end': 24.33096875,
  'text': ' अहाँ, अब बैठ गए'},
 {'speaker': 'SPEAKER_04',
  'start': 30.65909375,
  'end': 31.45221875,
  'text': 'แม่ฮาลี่'},
 {'speaker': 'SPEAKER_04',
  'start': 34.77659375,
  'end': 38.016593750000006,
  'text': ' और दो मिनेट रुख जाया जरकें तो पहले आई गए'},
 {'speaker': 'SPEAKER_00',
  'start': 3